# Preprocessing

## Setup

In [1]:
# Setup
import pandas as pd
import numpy as np
import os
import seaborn as sns
from datetime import datetime
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, Lasso, Ridge, ElasticNet
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
import matplotlib as plt
import seaborn as sns
import statsmodels.api as sm
from plotnine import ggplot, aes, geom_point
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import Lasso
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV

In [2]:
# Reading in data
ipeds = pd.read_excel("./IPEDS_data.xlsx")

In [3]:
# Viewing data
pd.set_option('display.max_columns', None)
ipeds.head()

,ID number,Name,year,ZIP code,Highest degree offered,County name,Longitude location of institution,Latitude location of institution,Religious affiliation,Offers Less than one year certificate,Offers One but less than two years certificate,Offers Associate's degree,Offers Two but less than 4 years certificate,Offers Bachelor's degree,Offers Postbaccalaureate certificate,Offers Master's degree,Offers Post-master's certificate,Offers Doctor's degree - research/scholarship,Offers Doctor's degree - professional practice,Offers Doctor's degree - other,Offers Other degree,Applicants total,Admissions total,Enrolled total,Percent of freshmen submitting SAT scores,Percent of freshmen submitting ACT scores,SAT Critical Reading 25th percentile score,SAT Critical Reading 75th percentile score,SAT Math 25th percentile score,SAT Math 75th percentile score,SAT Writing 25th percentile score,SAT Writing 75th percentile score,ACT Composite 25th percentile score,ACT Composite 75th percentile score,"Estimated enrollment, total","Estimated enrollment, full time","Estimated enrollment, part time","Estimated undergraduate enrollment, total","Estimated undergraduate enrollment, full time","Estimated undergraduate enrollment, part time","Estimated freshman undergraduate enrollment, total","Estimated freshman enrollment, full time","Estimated freshman enrollment, part time","Estimated graduate enrollment, total","Estimated graduate enrollment, full time","Estimated graduate enrollment, part time",Associate's degrees awarded,Bachelor's degrees awarded,Master's degrees awarded,Doctor's degrese - research/scholarship awarded,Doctor's degrees - professional practice awarded,Doctor's degrees - other awarded,Certificates of less than 1-year awarded,Certificates of 1 but less than 2-years awarded,Certificates of 2 but less than 4-years awarded,Postbaccalaureate certificates awarded,Post-master's certificates awarded,Number of students receiving an Associate's degree,Number of students receiving a Bachelor's degree,Number of students receiving a Master's degree,Number of students receiving a Doctor's degree,Number of students receiving a certificate of less than 1-year,Number of students receiving a certificate of 1 but less than 4-years,Number of students receiving a Postbaccalaureate or Post-master's certificate,Percent admitted - total,Admissions yield - total,"Tuition and fees, 2010-11","Tuition and fees, 2011-12","Tuition and fees, 2012-13","Tuition and fees, 2013-14",Total price for in-state students living on campus 2013-14,Total price for out-of-state students living on campus 2013-14,State abbreviation,FIPS state code,Geographic region,Sector of institution,Level of institution,Control of institution,Historically Black College or University,Tribal college,Degree of urbanization (Urban-centric locale),Carnegie Classification 2010: Basic,Total enrollment,Full-time enrollment,Part-time enrollment,Undergraduate enrollment,Graduate enrollment,Full-time undergraduate enrollment,Part-time undergraduate enrollment,Percent of total enrollment that are American Indian or Alaska Native,Percent of total enrollment that are Asian,Percent of total enrollment that are Black or African American,Percent of total enrollment that are Hispanic/Latino,Percent of total enrollment that are Native Hawaiian or Other Pacific Islander,Percent of total enrollment that are White,Percent of total enrollment that are two or more races,Percent of total enrollment that are Race/ethnicity unknown,Percent of total enrollment that are Nonresident Alien,Percent of total enrollment that are Asian/Native Hawaiian/Pacific Islander,Percent of total enrollment that are women,Percent of undergraduate enrollment that are American Indian or Alaska Native,Percent of undergraduate enrollment that are Asian,Percent of undergraduate enrollment that are Black or African American,Percent of undergraduate enrollment that are Hispanic/Latino,Percent of undergraduate enrollment that are Native Hawaiian or

### Dropping columns

In [4]:
# Dropping unnecessary columns
ipeds = ipeds.drop(columns = ['ID number','Name','year','ZIP code','County name',
                              'Longitude location of institution','Latitude location of institution','Offers Less than one year certificate',
                            'Offers One but less than two years certificate',"Offers Associate's degree", "Offers Two but less than 4 years certificate",
                             "Offers Bachelor's degree","Offers Postbaccalaureate certificate","Offers Master's degree","Offers Post-master's certificate",
                             "Offers Doctor's degree - research/scholarship","Offers Doctor's degree - other","Offers Other degree",
                             'FIPS state code','Sector of institution','Level of institution','Tribal college'])

### One hot encoding

In [5]:
# Unique value counts for categorical variables
[(i, len(ipeds[i].unique())) for i in ipeds.select_dtypes(include=['object', 'category']).columns]

[('Highest degree offered', 6),
 ('Religious affiliation', 55),
 ("Offers Doctor's degree - professional practice", 3),
 ('State abbreviation', 51),
 ('Geographic region', 9),
 ('Control of institution', 2),
 ('Historically Black College or University', 2),
 ('Degree of urbanization (Urban-centric locale)', 12),
 ('Carnegie Classification 2010: Basic', 9)]

In [6]:
# One hot encoding multicategory variables
ipeds = pd.get_dummies(ipeds, columns=['Highest degree offered', 
                                       'Religious affiliation',
                                       'Degree of urbanization (Urban-centric locale)',
                                      'Carnegie Classification 2010: Basic'], 
                       prefix=['Highest degree offered_',
                               'Religious affiliation_',
                               'Degree of urbanization_',
                              'Carnegie Classification_'])

ipeds = pd.get_dummies(ipeds, columns=['State abbreviation', 'Geographic region', 'Control of institution'])

In [7]:
# Binary variable reclassification (0,1)
ipeds["Offers Doctor's degree - professional practice"] = np.where(ipeds["Offers Doctor's degree - professional practice"]=="Yes", 1, 
                                                                   np.where(ipeds["Offers Doctor's degree - professional practice"]=='Implied no', 0, np.nan))

ipeds["Historically Black College or University"] = np.where(ipeds["Historically Black College or University"]=="Yes", 1, 
                                                                   np.where(ipeds["Historically Black College or University"]=='No', 0, np.nan))

ipeds['Control of institution_Private'] = np.where(ipeds['Control of institution']=='Public', 0, 1)


KeyError: 'Control of institution'

### Outlier detection

In [8]:
# Defining function to replace outliers with value of 95th or 5th percentile (high-low) in each feature

def replace_outliers(df):
    for column in df.columns:
        Q1 = df[column].quantile(0.25)
        Q3 = df[column].quantile(0.75)
        IQR = Q3 - Q1
        
        lower_bound = Q1 - 1.5 * IQR
        upper_bound = Q3 + 1.5 * IQR
        
        '''Compute the 5th and 95th percentile values'''
        lower_percentile_value = df[column].quantile(0.05)
        upper_percentile_value = df[column].quantile(0.95)
        
        '''Replace outliers with the respective percentile values'''
        df[column] = df[column].apply(lambda x: lower_percentile_value if x < lower_bound else upper_percentile_value if x > upper_bound else x)
    
    return df

# Replace outliers with the 5th and 95th percentile values
ipeds = replace_outliers(ipeds)


### Missing values

In [10]:
# Checking for missing values
missing_values_count = ipeds.isnull().sum() # generating array of missing values and columns
pd.set_option('display.max_rows', None) # setting view to all rows
missing_values_count = pd.DataFrame(missing_values_count).reset_index().rename(columns={"index":"column",0:"n"}).sort_values(by="n", ascending=False) # data
missing_values_count.loc[missing_values_count.n>0] # Show features with 1 or more missing values

,column,n
114,Endowment assets (year end) per FTE enrollment...,1018
10,SAT Writing 25th percentile score,820
11,SAT Writing 75th percentile score,820
93,Number of first-time undergraduates - in-state,623
98,Percent of first-time undergraduates - foreign...,623
97,Number of first-time undergraduates - foreign ...,623
96,Percent of first-time undergraduates - out-of-...,623
95,Number of first-time undergraduates - out-of-s...,623
94,Percent of first-time undergraduates - in-state,623
99,Number of first-time undergraduates - residenc...,623


In [11]:
# Dropping missing values rows for select variables
ipeds = ipeds.dropna(subset=['Admissions yield - total',
                            'Graduation rate - Bachelor degree within 4 years, total'])


In [12]:
# KNN imputation for continuous missing values
from sklearn.impute import KNNImputer

# Initialize KNNImputer with k=5 (you can adjust k as needed)
imputer = KNNImputer(n_neighbors=5)

# Perform KNN imputation on the entire DataFrame
ipeds_clean = pd.DataFrame(imputer.fit_transform(ipeds), columns=ipeds.columns)


### Exporting cleaned data

In [13]:
ipeds_clean.to_csv('ipeds_clean.csv', index=False)